In [1]:
import numpy as np
import theano
import theano.tensor as T
from triple_reader import triple_reader
# from question_reader_utf16 import question_reader
from question_reader_utf8 import question_reader
from gensim.models import word2vec
from nltk.corpus import stopwords
import sys

# initialize word2vec model
word_model = word2vec.Word2Vec.load_word2vec_format('word2vec.bin', binary=True)

numQuestions  2
[[u'The example in lines 4-8 primarily suggests that', u"Balzac's work was not especially popular among female readers", u'Balzac could not write convincingly about financial matters', u"Balzac's insights into character were not evident in his everyday life", u'people who knew Balzac personally could not respect him as an artist', u'readers had unreasonable expectation of Balzac the man'], [u"The author mentions Balzac's experience as a schoolboy in order to", u'explain why Balzac was unable to conduct his financial affairs properly', u"point out a possible source of Balzac's powerful imagination", u"exonerate the boarding school for Balzac's lackluster performance", u'foster the impression that Balzac was an unruly student', u"depict the conditions of boarding school life during Balzac's youth"]]


In [41]:
text_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/"
    "Senior Thesis Code/ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/4/Triples/4-long2.txt")

q_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/Senior Thesis Code/"
"ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/4/4-long2-q.txt")

a_file = ("/Users/SaahilM/Documents/Princeton/Academics/Thesis/Senior Thesis Code/"
"ModifiedEntityGraph/prod/MCTest/production/MCTest/OCR_text/4/4-long2-a.txt")

In [42]:
# read in file as tensors

tr = triple_reader(text_file)
# print tr.tripleList
tensor = tr.tensor

enMap = tr.enMap
relMap = tr.relMap

R = len(tensor)
N = len(tensor[0])
# dimension for encoding is arbitrary, we pick 20 here
d = 20

In [43]:
tensor_stack = np.hstack(tuple(tensor))
print(tensor_stack.shape)
print(N, N*R)

X = T.lmatrix('X')
q = T.lmatrix('q')
y = T.lmatrix('y')

(83, 4150)
83 4150


In [44]:
# Find the word in entity that's most similar to given word
# default topsim to lowest possible python int
def findSimEn(word):
    topsim = None
    topEn = None
    for en in enMap:
        if type(en) == int:
            continue
        try:
            sim = word_model.similarity(en, word)
        except KeyError as e:
            sim = None
        if sim > topsim:
            topsim = sim
            topEn = en
    return [topEn, topsim]

def findSimRel(word):
    topsim = None
    topRel = None
    for rel in relMap:
        if type(rel) == int:
            continue
        try:
            sim = word_model.similarity(rel, word)
        except KeyError as e:
            sim = None
        if sim > topsim:
            topsim = sim
            topRel = rel
    return [topRel, topsim]

# print findSimEn('critic')
# print findSimRel('go')

# Return top similarity, sim score, and isEn boolean
def findTopEnOrRel(word):
#     try:
    [topEn, topEnsim] = findSimEn(word)
    [topRel, topRelsim] = findSimRel(word)
    if topEnsim > topRelsim:
        return (topEn, topEnsim, True, word)
    else:
        return (topRel, topRelsim, False, word)
        
    # if can't find similarity, ignore it
#     except KeyError as e:
#         return [None, 0, False, word]
    
# Return top 2 sims for an array of words
def findTopEnOrRelArr(wordArr):
    topArrs = []
    for word in wordArr:
        top = findTopEnOrRel(word)
#         print top
        topArrs.append(top)
#     print topArrs
#     print topArrs
    sortedTop = sorted(topArrs, key=lambda x: -x[1] if x[1] is not None else sys.maxint)
#     print sortedTop
    top1 = sortedTop[0]
    top2 = None
    # if the top is a relation, we have to pick an entity
    if top1[2] == False:
        curIndex = 1
        while curIndex < len(sortedTop):
            cur = sortedTop[curIndex]
            if cur[2] == False:
                curIndex += 1
                continue
            else:
                top2 = cur
                break
    else:
        top2 = sortedTop[1]
    if top2 == None:
        top2 = [0, 0, True]
    return [top1, top2]

print(findTopEnOrRelArr(["critic", "went", "hi", "boat", "paint"]))

[('took', 0.5837877883692939, False, 'went'), ('wife', 0.19713489972182308, True, 'boat')]


In [45]:
qr = question_reader(q_file)

# print qr.numQuestions

numTrainQ = int(qr.numQuestions*(float(2)/3))
numTestQ = qr.numQuestions - numTrainQ
# print numTrainQ
# print numTestQ

CHOICES_PER_Q = 5

train_Q = []
test_Q = []
# VECTORIZE BY FINDING TOP TWO SIMILAR TO EACH QUESTION, AND MAKING A 2-HOT VECTOR OF LENGTH N+R
questions = qr.questionCombos
for i in xrange(qr.numQuestions):
    question = questions[i]
    question_words = question[0].split(" ")
    # remove stopwords
    question_words = [word for word in question_words if word not in stopwords.words('english')]
    [top1, top2] = findTopEnOrRelArr(question_words)
    print([top1, top2])
    
    # if word in question doesn't match, move on
    if top1[0] == None:
        continue
    
    # Now vectorize it to be of length N+R
    if top1[2] == True:
        index1 = enMap[top1[0]]
    else:
        index1 = N + relMap[top1[0]]
    if top2[2] == True:
        index2 = enMap[top2[0]]
    else:
        index2 = N + relMap[top2[0]]
    curQVec = np.zeros(N+R)
    curQVec[index1] = 1
    curQVec[index2] = 1
    if i < numTrainQ:
        train_Q.append(curQVec)
    else:
        test_Q.append(curQVec)
train_Q = np.array(train_Q)
train_Q = train_Q.T
test_Q = np.array(test_Q)
test_Q = test_Q.T
print(train_Q.shape)
print(test_Q.shape)

numQuestions  13
[('legislation', 0.48723350755038292, True, u'passage'), ('call', 0.34856425054760332, True, u'information')]
[('name', 0.39348277679045962, True, u'word'), ('year', 0.36010884827987122, True, u'nearly')]
[('Douglass', 1.0, True, u'Douglass'), ('newspapers', 0.77920511690145189, True, u'newspaper')]
[('Anthony', 0.99999999999999978, True, u'Anthony'), ('which', 0.59336037338987069, True, u'The')]
[('rights', 1.0, True, u'rights'), ('Douglass', 1.0, True, u'Douglass')]
[('name', 0.37368623494200653, True, u'phrase'), ('year', 0.36010884827987122, True, u'nearly')]
[('legislation', 0.48723350755038292, True, u'passage'), ('she', 0.35612284914044162, True, u'Lucy')]
[('Douglass', 1.0, True, u'Douglass'), ('Douglas', 1.0, True, u'Douglas')]
[('Douglass', 1.0, True, u'Douglass'), ('it', 0.49405084764469454, True, u'probably')]
[('rights', 1.0, True, u'rights'), ('Women', 0.50650276309085007, True, u"Women's")]
[('accepted', 0.45454538992104898, False, u'presented'), ('He', 

In [46]:
# Create a weight matrix of given size. 
# The matrix is initialized randomly with Gaussian distribution 
# with mean=0 and \sigma=0.1
def initializeWeightMatrix(in_size, out_size):
    return theano.shared(0.1 * np.random.randn(in_size, out_size))

# Create a bias vector of all zeros of given size
def initializeBiasVector(size):
    return theano.shared(np.zeros(size))

In [47]:
# Initialize all our parameters, given our dimensions.
# Input matrix has shape Nx(N*R)
# Query matrix has shape 5xnumQ
# A is the first matrix used to embed our input. It has size dxN
# B is the matrix used to embed the query. It has size dx(N+R)
# C is the next matrix used to embed our input. It has size dxN
# W is the final matrix. Takes output O and produces result w_embedded. It has size 5xd
# R is a matrix used to make the dimensions match from prior_Result. it has size dx5

def initializeParams(d, N):
    A = initializeWeightMatrix(d,N)
    B = initializeWeightMatrix(d,N+R)
    C = initializeWeightMatrix(d,N)
    W = initializeWeightMatrix(CHOICES_PER_Q,d)
    
#     A = theano.shared(initializeWeightMatrix(d, V))
#     B = theano.shared(initializeWeightMatrix(d, V))
#     C = theano.shared(initializeWeightMatrix(d, V))
#     W = theano.shared(initializeWeightMatrix(V, d))
    return A, B, C, W

A, B, C, W = initializeParams(d, N)
weightMatrices = [A, B, C, W]
print(W.shape.eval())

[ 5 20]


In [48]:
# Define the computational step
# Given input matrix X, query q, and weight matrices, we perform a computational step,
# also known as a "hop". Let M be the number of sentences.
# prior_result has dim dxnumQ
def hopComputation(prior_result, X, A, B, C):
    #m_i = Ax_i
    mem_matrix = A.dot(X) #dimension (dxN) x (Nx(NxR)) = dx(N*R)
    #u = Bq
#     u = B.dot(q) #dimension (dx(N+R)) x ((N+R)xnumQ) = dxnumQ
#     u = R.dot(prior_result.T) #dimension(dx5)x(5xnumQ) = dxnumQ
    u = prior_result
    #p_i = softmax(u^T m_i)
    probs = T.nnet.softmax(u.T.dot(mem_matrix)) #dimension(numQxd)x(dx(N*R)) = numQx(N*R)
    #C_i = Cx_i
    c = C.dot(X) #dimension (dxN) x (Nx(NxR)) = dx(N*R)
    o = c.dot(probs.T) #dimension (dx(N*R))x((N*R)xnumQ) = dxnumQ
    
    #w_embedded = Wo
#     w_embedded = W.dot(o).T #dimension (5xd)x(dxnumQ) = 5xnumQ.T = numQx5
    
#     result = T.nnet.softmax(w_embedded)
#     return w_embedded
    return o + u
    
    #output = sum of c_matrix * probs
#     o = (probs * c_embedded).sum(axis = 0)
    #result = 

In [49]:
initial_value = B.dot(q) #dimension dxnumQ
hops_result, updates = theano.scan(fn=hopComputation,
                                   n_steps=10,
                                   outputs_info=initial_value,
                                   non_sequences=[X] + [A,B,C],
                                   )
real_hops_result = hops_result[-1] #dimension dxnumQ
# Do W.dot(final_result)
w_embedded = W.dot(real_hops_result).T #dimension (5xd)x(dxnumQ) = 5xnumQ.T = numQx5
y_hat = T.nnet.softmax(w_embedded)
loss = T.nnet.categorical_crossentropy(y_hat, y).mean()

In [50]:
from __future__ import print_function

def inspect_inputs(i, node, fn):
    print(i, node, "input(s) value(s):", fn.inputs, end='')

def inspect_outputs(i, node, fn):
    print(" output(s) value(s):", fn.outputs)
    
def detect_nan(i, node, fn):
    for output in fn.outputs:
        if (not isinstance(output[0], np.random.RandomState) and
            np.isnan(output[0]).any()):
            print('*** NaN detected ***')
            theano.printing.debugprint(node)
            print('Inputs : %s' % [input[0] for input in fn.inputs])
            print('Outputs: %s' % [output[0] for output in fn.outputs])
            break

In [51]:
# Learning rate (chosen to be 0.01)
epsilon = 0.01

# This function trains our neural net, using stochastic gradient descent.
def train_MemNN(loss, X, q, y, y_hat):
    update_weights = []
    for weightMatrix in weightMatrices:
        update = T.grad(loss, weightMatrix)
        update_weights.append((weightMatrix, weightMatrix - update * epsilon))
    train_MemNN_func = theano.function(inputs=[X,q,y], outputs=[loss,y_hat], updates=update_weights, 
                        mode=theano.compile.MonitorMode(
#                             pre_func=inspect_inputs,
                            post_func=detect_nan))
    return train_MemNN_func

train_MemNN_func = train_MemNN(loss, X, q, y, y_hat)

In [52]:
def train_model(in_vect, question, answers, epochs=100):
    train_errors = []
    y_hats = []
    for i in xrange(epochs):
        error = 0
        [cur_loss, cur_yhat] = train_MemNN_func(in_vect, question, answers)
        error += cur_loss
#         print(error)
        train_errors.append(error)
        y_hats.append(cur_yhat)
    return [train_errors, y_hats]

def test_model(in_vect, question):
    # use a stub answers matrix, it doesn't really matter
    print(question.shape)
    [loss, y_hat] = train_MemNN_func(in_vect, question, np.zeros((len(question[0]), 5)).astype(int))
    return y_hat

In [53]:
in_vect = tensor_stack.astype(int)
question = train_Q.astype(int)

def letterToArr(letter):
    if letter == 'A':
        return np.array([1,0,0,0,0])
    elif letter == 'B':
        return np.array([0,1,0,0,0])
    elif letter == 'C':
        return np.array([0,0,1,0,0])
    elif letter == 'D':
        return np.array([0,0,0,1,0])
    elif letter == 'E':
        return np.array([0,0,0,0,1])

def convArrToArrs(arr):
    anz = []
    for i in xrange(len(arr)):
        anz.append(letterToArr(arr[i]))
    return np.array(anz)


az = open(a_file, "r")
anwz = []
for line in az:
    anwz.append(line.strip())
answers = convArrToArrs(anwz).astype(int)
train_ans = answers[:numTrainQ]
test_ans = anwz[numTrainQ:]
# print(answers)

# long1 = np.array([np.array([0,1,0,0,0]),
#                   np.array([1,0,0,0,0]),
#                   np.array([0,0,0,1,0]),
#                   np.array([0,0,0,0,1]),
#                   np.array([1,0,0,0,0]),
#                   np.array([1,0,0,0,0]),
#                   np.array([0,0,0,1,0]),
#                   np.array([0,0,0,1,0])
#                   ]).astype(int)

# print(in_vect)
# print(question)
# print(answers)
# print(type(in_vect[0][0]))

print(in_vect.shape)
print(question.shape)
print(answers.shape)
print(train_ans.shape)
# print(test_ans.shape)

num_iterations = 20
num_per_iter = 50
bestAns = []
mostCorrect = -1
for i in xrange(num_iterations):
    [train_errors, y_hats] = train_model(in_vect, question, train_ans, num_per_iter)
    res = test_model(in_vect, test_Q.astype(int))
#     print(res)
    predictions = []
    for ans in res:
        ind = np.argmax(ans)
        choice = None
        if ind == 0:
            choice = 'A'
        elif ind == 1:
            choice = 'B'
        elif ind == 2:
            choice = 'C'
        elif ind == 3:
            choice = 'D'
        elif ind == 4:
            choice = 'E'
        else:
            print('CANT GET HERE OHASDIIADSF')
        predictions.append(choice)
    numCorrect = 0
    for i in xrange(len(predictions)):
        if predictions[i] == test_ans[i]:
            numCorrect += 1
    if numCorrect > mostCorrect:
        mostCorrect = numCorrect
        bestAns = predictions
    
# print(train_errors)
# print(y_hats[-1])
print(test_ans)
print(bestAns)
print(mostCorrect)

(83, 4150)
(133, 8)
(13, 5)
(8, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
(133, 5)
['C', 'B', 'E', 'A', 'D']
['E', 'B', 'D', 'D', 'D']
2
